<a href="https://colab.research.google.com/github/YoonhoNam/hufsclass/blob/master/ch3_01_linear_regression_implementation_concise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3.3. Concise Implementation of Linear Regression

- PyTorch에서 제공하는 함수들을 활용하면 보다 간단하게 동일한 linear regression모델을 구현하고 훈련하는 과정을 확인해 볼 수 있습니다. 

In [0]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch import nn

In [0]:
# generating the synthetic dataset
def synthetic_data(w, b, num_examples):
  X = torch.zeros(size=(num_examples,len(w))).normal_(mean=0,std=1)
  y = torch.matmul(X, w) + b
  y += torch.zeros(size=y.shape).normal_(mean=0,std=0.01)
  return X, y
true_w = torch.tensor([[2],[-3.4]])
true_b = torch.tensor(4.2)
features, labels = synthetic_data(true_w, true_b, 1000)

In [0]:
# setting hyper-parameters
batch_size = 10
learning_rate = 0.03
num_epochs = 5

# defining data_iter, linreg. model, loss, sgd
data_iter = DataLoader(dataset=TensorDataset(features,labels),batch_size=batch_size,shuffle=True) # data_iter
linreg = nn.Linear(2,1,bias=True) # model
linreg.weight.data=torch.zeros(size=(1,2)).normal_(std=0.01) # parameter initialization (w)
linreg.bias.data=torch.zeros(size=(1,)) # parameter initialization (b)
loss = torch.nn.MSELoss() # loss
alg = torch.optim.SGD(linreg.parameters(),lr=learning_rate) # sgd

In [4]:
# training the model
for epoch in range(num_epochs):
  for X,y in data_iter:
    l=loss(linreg(X),y)
    alg.zero_grad()
    l.backward()
    alg.step()
  l_epoch = loss(linreg(features),labels)
  print('epoch %d, loss: %f' % (epoch, l_epoch))

w=list(linreg.parameters())[0][0]
print('Error in estimating w :',true_w.reshape(w.shape)-w)
b=list(linreg.parameters())[1][0]
print('Error in estimating b :',true_b-b)

epoch 0, loss: 0.000261
epoch 1, loss: 0.000096
epoch 2, loss: 0.000097
epoch 3, loss: 0.000096
epoch 4, loss: 0.000096
Error in estimating w : tensor([0.0009, 0.0002], grad_fn=<SubBackward0>)
Error in estimating b : tensor(0.0008, grad_fn=<SubBackward0>)


## Monitoring parameters
각각의 함수가 실제 어떤 역할을 하는지 또는 내가 원하는 기능을 제대로 수행하는 판단하는 직관적이면서도 쉬운 방법은 함수가 어떤 변화를 만들고 있는 지를 관찰하는 것입니다. 
각각의 함수의 뒤에서 model의 parameter중 하나를 출력하여 어떤 변화가 일어나는지 관찰해 봅시다.


In [ ]:
# monitring parameters
linreg.weight.data=torch.zeros(size=(1,2)).normal_(std=0.01) # parameter initialization (w)
linreg.bias.data=torch.zeros(size=(1,)) # parameter initialization (b)
alg.zero_grad()
for epoch in range(num_epochs):
  i=0
  for X,y in data_iter:
    i=i+1
    print('\n iteration #',i)
    print('#0 initial')
    print('b:', list(linreg.parameters())[1],'b_grad:',list(linreg.parameters())[1].grad)
    l=loss(linreg(X),y)
    print('#1 linreg(), loss()')
    print('b:', list(linreg.parameters())[1],'b_grad:',list(linreg.parameters())[1].grad)
    alg.zero_grad()
    print('#2 zero_grad()')
    print('b:', list(linreg.parameters())[1],'b_grad:',list(linreg.parameters())[1].grad)
    l.backward()
    print('#3 backward()')
    print('b:', list(linreg.parameters())[1],'b_grad:',list(linreg.parameters())[1].grad)
    alg.step()
    print('#4 step()')
    print('b:', list(linreg.parameters())[1],'b_grad:',list(linreg.parameters())[1].grad)
  break